In [ ]:
import cv2
import numpy as np
from keras.models import model_from_json
import time

# Load your trained model and weights
model_architecture = "C:\\Users\\VYOM\\Desktop\\ML_ASS\\model-bw1.json"
model_weights = "C:\\Users\\VYOM\\Desktop\\ML_ASS\\model-bw1.h5"

with open(model_architecture, "r") as json_file:
    loaded_model_json = json_file.read()
    model = model_from_json(loaded_model_json)
    model.load_weights(model_weights)

# Labels for gesture classes
class_labels = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ' ']

# Define the frame size (sz)
sz = 128  # You can adjust this size according to your camera and model requirements

# Set the desired size for displaying the webcam feed (lower resolution for smoother video)
display_width = 480
display_height = 480

# Define the region for gesture recognition (top-right corner)
region_x = display_width - 100  # Adjust the position as needed
region_y = 0
region_width = 300
region_height = 300

# Create a colored boundary for the gesture region
boundary_color = (0, 255, 0)  # Green color

# Create a function to perform real-time gesture recognition
def predict_gesture_from_webcam():
    cap = cv2.VideoCapture(0)  # Open the webcam (you may need to adjust the camera index)
    cap.set(3, 640)  # Lower the display width resolution
    cap.set(4, 480)  # Lower the display height resolution
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Error: Unable to capture video.")
            break

        # Crop the region for gesture recognition
        gesture_region = frame[region_y:region_y + region_height, region_x:region_x + region_width]

        # Convert the cropped region to grayscale
        gesture_region = cv2.cvtColor(gesture_region, cv2.COLOR_BGR2GRAY)
        gesture_region = cv2.resize(gesture_region, (sz, sz))  # Resize the frame to match the model's input size
        gesture_region = np.expand_dims(gesture_region, axis=0)  # Add batch dimension
        gesture_region = np.expand_dims(gesture_region, axis=-1)  # Add channel dimension
        gesture_region = gesture_region / 255.0  # Normalize pixel values

        # Predict the gesture
        prediction = model.predict(gesture_region)

        # Get the predicted label
        predicted_label = class_labels[np.argmax(prediction)]

        # Display the predicted label on the green boundary box
        text_x = region_x
        text_y = region_y + 30
        cv2.putText(frame, "Predicted: " + predicted_label, (text_x, text_y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

        # Draw a colored boundary for the gesture region (top-right corner)
        cv2.rectangle(frame, (region_x, region_y), (region_x + region_width, region_y + region_height), boundary_color, 2)

        # Resize the frame to display size
        frame = cv2.resize(frame, (display_width, display_height))

        cv2.imshow('Webcam Feed', frame)

        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break

    # Release the camera capture object
    cap.release()
    cv2.destroyAllWindows()

# Call the function for real-time prediction
predict_gesture_from_webcam()
